In [15]:
import neat
import os
import evogym.envs
from evogym import is_connected, has_actuator, get_full_connectivity, hashable
import numpy as np
from pathos.multiprocessing import ProcessPool
import errno
import dill
import sys
sys.path.append('../')

import neat
from hyperneat.new_hyperNEAT import create_phenotype_network
from hyperneat.substrate import Substrate
from hyperneat.visualize import draw_net

import itertools as it
import time
import imageio

import sgr_main as hyper
from matplotlib import pyplot as plt


In [19]:
def get_robot(genome, params, neat_config):
    cppn = neat.nn.FeedForwardNetwork.create(genome, neat_config)

    design_substrate = hyper.morph_substrate(params)
    design_net = hyper.create_phenotype_network(cppn, design_substrate)

    robot = hyper.generate_robot(design_net, params["robot_size"])
    if not hyper.eval_genome_constraint(robot):
        return robot, None
    connections = get_full_connectivity(robot)
    env = evogym.envs.gym.make(params["env"], body=robot, connections=connections)
    img = env.render(mode='img')
    env.close()
    return robot, img

In [20]:
ENV = "Walker-v0" # env_names = ["CaveCrawler-v0", "UpStepper-v0", "ObstacleTraverser-v0"]
STEPS = 500
ROBOT_SIZE = 5
POP = 64

params = {"robot_size": ROBOT_SIZE, "env": ENV, "steps": STEPS, "pop_size": POP}

neat_config = neat.config.Config(
    neat.genome.DefaultGenome, 
    neat.reproduction.DefaultReproduction, 
    neat.species.DefaultSpeciesSet, 
    neat.stagnation.DefaultStagnation, 
    "../configs/hyperNEAT_config"
)


In [21]:
directory = "results/walker/"
file = "run_4_pop.pkl"
with open(f"../{directory}{file}", "rb") as file:
    pop = dill.load(file)
    
champion = pop.best_genome

In [22]:
# hyper.single_genome_fit(champion, params, neat_config, True)
params["cpu"]= 4
genomes = [(g.key, g) for g in pop.population.values()]
# hyper.fit_func(genomes, neat_config, params)

In [23]:
for k, v in pop.species.species.items():
    print("Specie ", k)
    for g in v.members.values():
        if g.fitness != None:
            # hyper.single_genome_fit(
            #     g, 
            #     params, 
            #     neat_config, 
            #     True
            # )
            r, img = get_robot(g, params, neat_config)
            if img is not None:
                plt.imshow(img)
                plt.show()


Specie  6


AttributeError: module 'sgr_main' has no attribute 'morph_substrate'

In [ ]:
hyper.single_genome_fit(champion, params, neat_config, True)